In [1]:
import sys
sys.path.insert(0, '/home/ldoyle/packages')
import h5py
import numpy as np
from tqdm import tqdm
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import pysindy as ps
from lr_ed import localreg
from datetime import datetime
from scipy.fft import fft, fftfreq,rfft2

import scipy.signal as signal
from obspy.signal import filter as obsfilt

In [2]:
import glob
file_list = glob.glob("/home/ldoyle/notebooks/rad_4_smooth_data/*")

In [3]:
file_list

['/home/ldoyle/notebooks/rad_4_smooth_data/gaussian_smoothed_channels_7500_12500_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy',
 '/home/ldoyle/notebooks/rad_4_smooth_data/gaussian_smoothed_channels_7500_12500_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231214Z.npy',
 '/home/ldoyle/notebooks/rad_4_smooth_data/gaussian_smoothed_channels_7500_12500_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231314Z.npy',
 '/home/ldoyle/notebooks/rad_4_smooth_data/gaussian_smoothed_channels_7500_12500_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231414Z.npy',
 '/home/ldoyle/notebooks/rad_4_smooth_data/gaussian_smoothed_channels_7500_12500_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231514Z.npy',
 '/home/ldoyle/notebooks/rad_4_smooth_data/gaussian_smoothed_channels_7500_12500_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231614Z.npy',
 '/home/ldoyle/notebooks/rad_4_smooth_data/gaussian_smoothed_channels_7500_12500_South-C1-LR-95km-P1kHz-GL50m-SP

In [4]:
time_subsample = 1
full_dat = np.load(file_list[0])#data[7500:12500,::time_subsample]

for i in np.arange(1,len(file_list)):
    tmp=np.load(file_list[i])
    print(tmp.shape)
    full_dat = np.hstack((full_dat,tmp))

(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)
(5000, 12000)


In [5]:
full_dat.shape

(5000, 228000)

In [6]:
fk_dat = np.fft.fftn(full_dat)

xf = np.fft.fftfreq(5000, d=2)

In [7]:
tf = np.fft.fftfreq(full_dat.shape[1], d=time_subsample/200)#[:N//2]

X, Y = np.meshgrid(tf,xf)

In [ ]:
plt.figure(figsize=(6,10))
cp = plt.contourf(X[:,:X.shape[1]//2], Y[:,:X.shape[1]//2], 10*np.log10(np.abs(fk_dat[:,:X.shape[1]//2])**2))

plt.scatter(np.sqrt(9.8*np.linspace(-0.01,0.01,30)*np.tanh(142*np.linspace(-0.01,0.01,30))),
            np.linspace(-0.01,0.01,30), c="k")
# plt.scatter(-np.sqrt(9.8*np.linspace(-0.01,0.01,30)*np.tanh(50*np.linspace(-0.01,0.01,30))),
#             np.linspace(-0.01,0.01,30), c="k")
plt.xlim(-0.,0.2)
plt.ylim(-0.01,0.01)
plt.xlabel("Freq. (Hz)")
plt.ylabel("Wavenumber (1/m)",size=12)
plt.colorbar(cp, pad = 0.04)
plt.show()